# Análise de Candidatos ao Vestibular da UEA

-  Análise exploratória
    - [X] Verificar existência de dados nulos
    - [X] Formatar ``datanasc`` de str para datetime
    - [X] Formatar ``classificacao`` de float para int
    - [X] Criar atributo ``idade``
    - [X] Formatar atributo ``id_referencia_grupo``
    - [ ] Ordenar (na ordem que aparece na leitura do PDF) dataframe considerando: 
        1. Por ``ano``
        2. Por ``curso_nome_referencia``
        3. Por ``id_curso_referencia``
        4. Por ``classificacao`` 


## Pré-processamento dos dados

In [205]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import euclidean

from datetime import datetime, date

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

### Carregando o dataset

In [206]:
data = pd.read_csv('../data/csv/candidates_info.csv', sep=',', encoding='utf-8')

In [207]:
data.head(5)

,numinscricao,nome,datanasc,classificacao,opcao_curso_2,opcao_3_curso,habilidades,nota_final,nota_cg,nota_etapa_1,disc1,disc2,disc3,nota_ce,nota_redacao,nota_etapa_2,opcao_curso_1,id_referencia_curso,nome_referencia_curso,ano
0,6354130-0,EDIJANE PINHEIRO MARTINS,04/03/92,313.0,No,No,No,"31,161","19,048","19,048",3,5,6,"28,00","15,27","43,273",4069,4069,Odontologia - Integral - Manaus,2019
1,6318761-2,JOEL DOS SANTOS DE SOUZA,09/03/97,314.0,No,No,No,"31,154","29,762","29,762",3,1,4,"16,00","16,55","32,545",4069,4069,Odontologia - Integral - Manaus,2019
2,6379123-4,CARINA DAS CHAGAS SEIXAS,23/10/96,315.0,No,No,No,"31,154","29,762","29,762",3,3,2,"16,00","16,55","32,545",4069,4069,Odontologia - Integral - Manaus,2019
3,6549853-4,IRLA DA SILVA PEREIRA,07/02/01,316.0,No,No,No,"31,113","30,952","30,952",4,0,4,"16,00","15,27","31,273",4069,4069,Odontologia - Integral - Manaus,2019
4,6393350-0,IVANIR LIMA PEREIRA,11/03/91,317.0,No,No,No,"31,113","30,952","30,952",2,3,3,"16,00","15,27","31,273",4069,4069,Odontologia - Integral - Manaus,2019


## Análise exploratória dos dados

In [208]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184389 entries, 0 to 184388
Data columns (total 20 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   numinscricao           184389 non-null  object 
 1   nome                   184389 non-null  object 
 2   datanasc               184389 non-null  object 
 3   classificacao          184389 non-null  float64
 4   opcao_curso_2          184389 non-null  object 
 5   opcao_3_curso          184389 non-null  object 
 6   habilidades            184389 non-null  object 
 7   nota_final             184389 non-null  object 
 8   nota_cg                184389 non-null  object 
 9   nota_etapa_1           184389 non-null  object 
 10  disc1                  184389 non-null  int64  
 11  disc2                  184389 non-null  int64  
 12  disc3                  184389 non-null  int64  
 13  nota_ce                184389 non-null  object 
 14  nota_redacao           184389 non-nu

#### Verificando a existência de colunas com dados nulos

In [209]:
data.isnull().sum()

numinscricao             0
nome                     0
datanasc                 0
classificacao            0
opcao_curso_2            0
opcao_3_curso            0
habilidades              0
nota_final               0
nota_cg                  0
nota_etapa_1             0
disc1                    0
disc2                    0
disc3                    0
nota_ce                  0
nota_redacao             0
nota_etapa_2             0
opcao_curso_1            0
id_referencia_curso      0
nome_referencia_curso    0
ano                      0
dtype: int64

#### Formatando coluna 'datanasc' para o formato date

In [210]:
data['datanasc'] = pd.to_datetime(data['datanasc'], format='%d/%m/%y')
data.head(4)

,numinscricao,nome,datanasc,classificacao,opcao_curso_2,opcao_3_curso,habilidades,nota_final,nota_cg,nota_etapa_1,disc1,disc2,disc3,nota_ce,nota_redacao,nota_etapa_2,opcao_curso_1,id_referencia_curso,nome_referencia_curso,ano
0,6354130-0,EDIJANE PINHEIRO MARTINS,1992-03-04,313.0,No,No,No,"31,161","19,048","19,048",3,5,6,"28,00","15,27","43,273",4069,4069,Odontologia - Integral - Manaus,2019
1,6318761-2,JOEL DOS SANTOS DE SOUZA,1997-03-09,314.0,No,No,No,"31,154","29,762","29,762",3,1,4,"16,00","16,55","32,545",4069,4069,Odontologia - Integral - Manaus,2019
2,6379123-4,CARINA DAS CHAGAS SEIXAS,1996-10-23,315.0,No,No,No,"31,154","29,762","29,762",3,3,2,"16,00","16,55","32,545",4069,4069,Odontologia - Integral - Manaus,2019
3,6549853-4,IRLA DA SILVA PEREIRA,2001-02-07,316.0,No,No,No,"31,113","30,952","30,952",4,0,4,"16,00","15,27","31,273",4069,4069,Odontologia - Integral - Manaus,2019


In [211]:
data['classificacao'] = data['classificacao'].astype(int)

In [212]:
data.sample(2)

,numinscricao,nome,datanasc,classificacao,opcao_curso_2,opcao_3_curso,habilidades,nota_final,nota_cg,nota_etapa_1,disc1,disc2,disc3,nota_ce,nota_redacao,nota_etapa_2,opcao_curso_1,id_referencia_curso,nome_referencia_curso,ano
13513,6593999-9,BERNARDO SILVA VIANA,2002-01-30,237,07067,No,No,"80,913","76,190","76,190",10,10,12,"64,00","21,64","85,636",6067,7067,Medicina - Integral - Manaus,2019
166737,2367167-0,CAROLINE MOURA CALHEIROS,1999-02-12,130,No,No,No,"39,468","28,571","28,571",2,6,7,"30,00","20,36","50,364",3088,3088,Direito - Noturno - Manacapuru,2022


## Criando novos atributos

In [213]:
def calculate_age(row) -> int:
    born = row['datanasc']
    year = row['ano']
    age = year - born.year
    return age

In [214]:
data['idade'] = data.apply(calculate_age, axis=1)
data.drop(columns=['datanasc'], axis=1, inplace=True)
data.head(5)

,numinscricao,nome,classificacao,opcao_curso_2,opcao_3_curso,habilidades,nota_final,nota_cg,nota_etapa_1,disc1,disc2,disc3,nota_ce,nota_redacao,nota_etapa_2,opcao_curso_1,id_referencia_curso,nome_referencia_curso,ano,idade
0,6354130-0,EDIJANE PINHEIRO MARTINS,313,No,No,No,"31,161","19,048","19,048",3,5,6,"28,00","15,27","43,273",4069,4069,Odontologia - Integral - Manaus,2019,27
1,6318761-2,JOEL DOS SANTOS DE SOUZA,314,No,No,No,"31,154","29,762","29,762",3,1,4,"16,00","16,55","32,545",4069,4069,Odontologia - Integral - Manaus,2019,22
2,6379123-4,CARINA DAS CHAGAS SEIXAS,315,No,No,No,"31,154","29,762","29,762",3,3,2,"16,00","16,55","32,545",4069,4069,Odontologia - Integral - Manaus,2019,23
3,6549853-4,IRLA DA SILVA PEREIRA,316,No,No,No,"31,113","30,952","30,952",4,0,4,"16,00","15,27","31,273",4069,4069,Odontologia - Integral - Manaus,2019,18
4,6393350-0,IVANIR LIMA PEREIRA,317,No,No,No,"31,113","30,952","30,952",2,3,3,"16,00","15,27","31,273",4069,4069,Odontologia - Integral - Manaus,2019,28


In [215]:
data.sample(8)

,numinscricao,nome,classificacao,opcao_curso_2,opcao_3_curso,habilidades,nota_final,nota_cg,nota_etapa_1,disc1,disc2,disc3,nota_ce,nota_redacao,nota_etapa_2,opcao_curso_1,id_referencia_curso,nome_referencia_curso,ano,idade
17992,6351551-2,FLAVIA ADILA PEREIRA PINTO,397,06069,07069,No,"40,320","38,095","38,095",4,4,5,"26,00","16,55","42,545",5069,7069,Odontologia - Integral - Manaus,2019,20
152129,2419813-7,ALINE BIANCA DE SOUZA FACANHA,54,02032,03032,No,"30,492","22,619","22,619",3,2,4,"18,00","20,36","38,364",1032,2032,Licenciatura em Ciências Biológicas - Vesperti...,2022,20
123163,1185344-1,VALDENILDO NATIVIDADE SANTOS,526,02079,03079,No,"27,093","13,095","13,095",3,7,1,"22,00","19,09","41,091",1079,3079,Direito - Noturno - Parintins,2021,20
50400,6370650-4,BRUNO EDUARDO FEITOSA DO NASCIMENTO,149,02027,03027,No,"39,916","39,286","39,286",6,2,4,"24,00","16,55","40,545",1027,1027,Licenciatura em Ciências Biológicas - Matutino...,2019,18
114336,1176543-7,LUCAS GUIMARAES DE FREITAS,185,02088,03088,No,"29,682","25,000","25,000",2,3,2,"14,00","20,36","34,364",1088,2088,Direito - Noturno - Eirunepé,2021,18
160989,2339980-5,MARCOS ANTONIO SILVA DE MELO,353,03018,No,No,"41,056","40,476","40,476",2,3,5,"20,00","21,64","41,636",2018,3018,Engenharia da Computação - Integral - Manaus,2022,19
166535,2369986-8,IZANEIDE RODRIGUES CAETANO,5,No,No,No,"35,699","29,762","29,762",3,4,3,"20,00","21,64","41,636",3085,3085,Tecnologia em Gestão de Turismo - Matutino - C...,2022,28
68190,8288381-5,KATIA EDILENE PEREIRA BAKER,50,02011,03011,No,"35,195","28,571","28,571",2,1,9,"24,00","17,82","41,818",1011,1011,Ciências Contábeis - Noturno - Manaus,2020,27


#### Padronizando as colunas referentes a opção de curso

In [216]:
data['opcao_curso_1'] = data['opcao_curso_1'].astype(str)
data['id_referencia_curso'] = data['id_referencia_curso'].astype(str)

In [217]:
data['opcao_3_curso'] = data['opcao_3_curso'].apply(lambda x: x[1:] if str(x).startswith('0') else x)
data['opcao_curso_2'] = data['opcao_curso_2'].apply(lambda x: x[1:] if str(x).startswith('0') else x)

In [218]:
data.head(8)

,numinscricao,nome,classificacao,opcao_curso_2,opcao_3_curso,habilidades,nota_final,nota_cg,nota_etapa_1,disc1,disc2,disc3,nota_ce,nota_redacao,nota_etapa_2,opcao_curso_1,id_referencia_curso,nome_referencia_curso,ano,idade
0,6354130-0,EDIJANE PINHEIRO MARTINS,313,No,No,No,"31,161","19,048","19,048",3,5,6,"28,00","15,27","43,273",4069,4069,Odontologia - Integral - Manaus,2019,27
1,6318761-2,JOEL DOS SANTOS DE SOUZA,314,No,No,No,"31,154","29,762","29,762",3,1,4,"16,00","16,55","32,545",4069,4069,Odontologia - Integral - Manaus,2019,22
2,6379123-4,CARINA DAS CHAGAS SEIXAS,315,No,No,No,"31,154","29,762","29,762",3,3,2,"16,00","16,55","32,545",4069,4069,Odontologia - Integral - Manaus,2019,23
3,6549853-4,IRLA DA SILVA PEREIRA,316,No,No,No,"31,113","30,952","30,952",4,0,4,"16,00","15,27","31,273",4069,4069,Odontologia - Integral - Manaus,2019,18
4,6393350-0,IVANIR LIMA PEREIRA,317,No,No,No,"31,113","30,952","30,952",2,3,3,"16,00","15,27","31,273",4069,4069,Odontologia - Integral - Manaus,2019,28
5,6612916-8,LAISA DOS SANTOS GALVAO,318,No,No,No,"31,046","25,000","25,000",4,2,3,"18,00","19,09","37,091",4069,4069,Odontologia - Integral - Manaus,2019,16
6,6424859-3,ELCIA NEVES DOS SANTOS,319,No,No,No,"31,046","25,000","25,000",5,3,1,"18,00","19,09","37,091",4069,4069,Odontologia - Integral - Manaus,2019,18
7,6386972-1,LAIZA DE FARIAS MAKLOUF,320,No,No,No,"31,004","26,190","26,190",4,3,2,"18,00","17,82","35,818",4069,4069,Odontologia - Integral - Manaus,2019,17


In [219]:
data.dtypes

numinscricao             object
nome                     object
classificacao             int32
opcao_curso_2            object
opcao_3_curso            object
habilidades              object
nota_final               object
nota_cg                  object
nota_etapa_1             object
disc1                     int64
disc2                     int64
disc3                     int64
nota_ce                  object
nota_redacao             object
nota_etapa_2             object
opcao_curso_1            object
id_referencia_curso      object
nome_referencia_curso    object
ano                       int64
idade                     int64
dtype: object

## Relacionado as informações sobre o curso com cada candidato

Seguindo o padrão observado na extração de dados, id_referencia curso é composto por **XXYYY**. Onde:
- XX: grupo relacionado ao curso
- YY: código do curso

Para conseguir comparar o dataset de candidatos com o de vagas, iremos utilizar somente as informações do grupo (XX) na coluna: 

In [220]:
data['id_referencia_curso'] = data['id_referencia_curso'].apply(lambda x: x[:1])

In [221]:
data.sample(4)

,numinscricao,nome,classificacao,opcao_curso_2,opcao_3_curso,habilidades,nota_final,nota_cg,nota_etapa_1,disc1,disc2,disc3,nota_ce,nota_redacao,nota_etapa_2,opcao_curso_1,id_referencia_curso,nome_referencia_curso,ano,idade
31455,6654479-3,HUGO HENRIQUE AMORIM BURGOS FELIX,390,2018,3018,No,"38,403","35,714","35,714",3,2,6,"22,00","19,09","41,091",1018,3,Engenharia da Computação - Integral - Manaus,2019,16
301,6371358-6,IRILENE DO SOCORRO OLIVEIRA DE QUEIROZ,133,6016,7016,No,"46,528","45,238","45,238",3,4,8,"30,00","17,82","47,818",5016,5,Enfermagem - Integral - Manaus,2019,19
21441,6351456-7,EDILSON MIGUEL PINTO,119,2052,3052,No,"23,996","23,810","23,810",4,1,2,"14,00","10,18","24,182",1052,2,Licenciatura em Letras - Língua Portuguesa - V...,2019,23
142525,2382303-8,LIDIA RABELO ARAUJO,6,2029,3029,No,"47,909","50,000","50,000",6,1,7,"28,00","17,82","45,818",1029,1,Licenciatura em Ciências Biológicas - Vesperti...,2022,20


Agora, vamos ordenar o dataset por ano, id_referencia_curso e classificação

In [222]:
data.sample(10)

,numinscricao,nome,classificacao,opcao_curso_2,opcao_3_curso,habilidades,nota_final,nota_cg,nota_etapa_1,disc1,disc2,disc3,nota_ce,nota_redacao,nota_etapa_2,opcao_curso_1,id_referencia_curso,nome_referencia_curso,ano,idade
39770,6785510-5,IVANETE BRAZIL DE SOUZA,157,2062,3062,No,"28,582","22,619","22,619",3,3,3,"18,00","16,55","34,545",1062,3,Licenciatura em Pedagogia - Matutino - Tabatinga,2019,30
127749,1343503-5,CARLOS EDUARDO FRANCO RIBEIRO,153,6037,7037,No,"21,820","13,095","13,095",3,3,1,"14,00","16,55","30,545",5037,5,Licenciatura em Educação Física - Vespertino -...,2021,19
33044,6822507-5,DANIEL ARAUJO BELEM,77,3025,No,No,"49,056","40,476","40,476",7,2,9,"36,00","21,64","57,636",2025,3,Engenharia Mecânica - Integral - Manaus,2019,20
111389,1183999-6,MAGSON SILVA DO NASCIMENTO,9,2052,3052,No,"32,344","32,143","32,143",4,2,2,"16,00","16,55","32,545",1052,2,Licenciatura em Matemática - Vespertino - Taba...,2021,39
13982,6543559-1,JOAO PEDRO ROSA BARRONCAS,706,6067,7067,No,"70,689","64,286","64,286",9,9,11,"58,00","19,09","77,091",5067,7,Medicina - Integral - Manaus,2019,17
59245,6808293-2,DEBORA CAMILE FERREIRA ARAUJO,77,2014,3014,No,"63,903","60,714","60,714",8,9,7,"48,00","19,09","67,091",1014,2,Direito - Noturno - Manaus,2019,17
97892,8578148-7,GABRIEL DA SILVA VIEIRA,2,6063,7063,No,"26,665","20,238","20,238",3,1,3,"14,00","19,09","33,091",5063,7,Medicina - Integral - Manaus,2020,23
121578,1167017-7,EZEQUIEL DE SOUZA GOMES,48,2070,3070,No,"27,963","27,381","27,381",2,2,2,"12,00","16,55","28,545",1070,3,Licenciatura em História - Noturno - Parintins,2021,29
182485,2407627-9,LAIRA BALIEIRO LUCAS,27,No,No,No,"32,451","23,810","23,810",2,4,5,"22,00","19,09","41,091",8015,8,Direito - Vespertino - Manaus,2022,18
46963,6386889-0,JULIA VITORIA DA SILVA MARTINS,174,2015,3015,No,"53,240","53,571","53,571",4,5,6,"30,00","22,91","52,909",1015,1,Direito - Vespertino - Manaus,2019,17


In [223]:
dados_vaga = pd.read_csv('../data/csv/course_info.csv', sep=',', encoding='utf-8')

In [224]:
def verificar_aprovacao(row, dados_vaga) -> bool:
    for _, info_vaga in dados_vaga.iterrows():
        if row['ano'] == info_vaga['ano']:
            for opcao_curso in [row['opcao_curso_1'], row['opcao_curso_2'], row['opcao_3_curso']]:
                if str(opcao_curso).split()[0] == str(info_vaga['grupo']) and \
                   row['nome_referencia_curso'] == info_vaga['nome_curso']:
                    if row['classificacao'] <= info_vaga['qtde_vagas']:
                        print("Candidato:", row['nome'], "foi selecionado!")
                        return True
    print("Candidato:", row['nome'], "não foi selecionado.")
    return False

In [229]:
# Aplicar a função ao dataset de candidatos
data['passou'] = data.apply(lambda row: verificar_aprovacao(row, dados_vaga), axis=1)

Candidato: EDIJANE PINHEIRO MARTINS não foi selecionado.
Candidato: JOEL DOS SANTOS DE SOUZA não foi selecionado.
Candidato: CARINA DAS CHAGAS SEIXAS não foi selecionado.
Candidato: IRLA DA SILVA PEREIRA não foi selecionado.
Candidato: IVANIR LIMA PEREIRA não foi selecionado.
Candidato: LAISA DOS SANTOS GALVAO não foi selecionado.
Candidato: ELCIA NEVES DOS SANTOS não foi selecionado.
Candidato: LAIZA DE FARIAS MAKLOUF não foi selecionado.
Candidato: CAMILA PANTOJA QUADRO não foi selecionado.
Candidato: SUELY SOUZA DE ALMEIDA não foi selecionado.
Candidato: ISLAM DA SILVA SOARES não foi selecionado.
Candidato: KATSUANE IJUMA COSTA não foi selecionado.
Candidato: OTAVIO AUGUSTO FERREIRA BARBOSA não foi selecionado.
Candidato: EVERSON DE SOUZA TAVARES não foi selecionado.
Candidato: SAMUELY FIRMINO PEREIRA não foi selecionado.
Candidato: SARA CARVALHO DA SILVA não foi selecionado.
Candidato: DHEYNICCE SILVA DA COSTA não foi selecionado.
Candidato: MARIA ALICE DE ARAUJO DE AIRAO não foi s

KeyboardInterrupt: 

In [ ]:
""" dados_vaga = pd.read_csv('../data/csv/course_info.csv', sep=',', encoding='utf-8')
data['passou'] = data.apply(lambda row: verificar_aprovacao(row, dados_vaga), axis=1) """

: 